# Downloading and coverting ERA5 files

Information on the fifth generation ECMWF atmospheric reanalysis of the global climate ERA5 is found at:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

# 1) Requesting ERA5 data

a) Go to the ERA5 hourly data on single levels from 1979 to present (https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form)

b) In product type select Reanalysis

c) In Variable select Popular -> 10m u-component of wind, 10m v-component of wind, and Mean sea level pressure.

d) In this example we will prepare the data for Hurricane Arthur, from 06/27/2014 to 07/09/2014. Therefore, we will have to download 2 ERA5 files, one from 06/27/2014 to 06/30/2014 and another from 07/01/2014 to 07/09/2014. 

First, we will download the data from 06/27/2014 to 06/30/2014. To do that we just need to select the Year, Month and Day accordingly. We will work with inputs every 6 hours, so select the Time: 00:00, 06:00, 12:00, and 18:00.

e) In the Geographical area tab select “Whole available region”

f) Select the GRIB format in the Format tab.

g) Click on “Show API request” and save the code.


# 2) Downloading ERA5 data

Importing the necessary lybraries:


cfgrib is installed as:
conda install -c conda-forge cfgrib
xarray is installed as:
conda install -c conda-forge xarray dask netCDF4 bottleneck

In [15]:
import sys;sys.path.append('../adcirc_swan')
import adcirc as adc;import regional as reg
import utils as ut
import sys,getopt
import pandas as pd
import numpy as np
import pathlib as pl
import numpy.core.multiarray 
import netCDF4 as nc4
import xarray as xr
import scipy.interpolate
import datetime
import cdsapi
import matplotlib as mpl
from PIL import Image
from datetime import timedelta
from mpl_toolkits.basemap import Basemap
import glob
import os
import matplotlib.tri as tri
import datetime
import matplotlib.pyplot as plt
from datetime import timedelta
c = cdsapi.Client()

Paste the API request and start downloading:

In [20]:
file1 = pd.read_csv('/Users/tmiesse/work/FHRL/eeslr/field/rbr/cbec/ruskin_process/cbec_s1_204010_20221012_1354_hs.csv',skiprows=1)
file2 = pd.read_csv('/Users/tmiesse/Downloads/CO-OPS__8575512__bp.csv')
start_h0 = pd.to_datetime(file1['Time'][0]);
end_h0   = pd.to_datetime(file1['Time'].iloc[-1]);

nearest = np.where((start_h0-timedelta(minutes=35)<=pd.to_datetime(file2['Date Time']))&(pd.to_datetime(file2['Date Time'])<(start_h0+timedelta(minutes=35))))


In [22]:
pd.to_datetime(file2['Date Time'])

0      2022-05-06 00:00:00
1      2022-05-06 00:06:00
2      2022-05-06 00:12:00
3      2022-05-06 00:18:00
4      2022-05-06 00:24:00
               ...        
7435   2022-06-05 23:30:00
7436   2022-06-05 23:36:00
7437   2022-06-05 23:42:00
7438   2022-06-05 23:48:00
7439   2022-06-05 23:54:00
Name: Date Time, Length: 7440, dtype: datetime64[ns]

In [23]:
start_h0

Timestamp('2022-03-07 13:45:00')

## ESO dummy correct parse calc.

In [11]:
dummy = 21000000
minute = 3
sec = 36
time = (minute*60)+sec
dps = dummy/time
print(dps)

97222.22222222222


In [5]:
days = np.arange(5,20).astype(str).tolist()


In [6]:
time = ['00:00', '01:00', '02:00','03:00', '04:00', '05:00',
        '06:00', '07:00', '08:00','09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00','15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00','21:00', '22:00', '23:00',]
        #
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': [
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
            'Mean_sea_level_pressure',
            #'Sea_Ice_Concentration',
        ],
        'year': '2022',
        'month': '09',
        'day': days,
        'time': time,
    },
    'download.grib')

2022-09-20 09:48:19,079 INFO Welcome to the CDS
2022-09-20 09:48:19,080 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-09-20 09:48:21,273 INFO Request is queued
2022-09-20 09:48:23,171 INFO Request is failed
2022-09-20 09:48:23,172 ERROR Message: the request you have submitted is not valid
2022-09-20 09:48:23,172 ERROR Reason:  Mars server task finished in error; UserError: Restricted access to ERA5T. Please, check that your date selection is valid. For more information, visit https://climate.copernicus.eu/climate-reanalysis [mars]; Error code is -2; Request failed; Some errors reported (last error -2)
2022-09-20 09:48:23,173 ERROR   Traceback (most recent call last):
2022-09-20 09:48:23,174 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 59, in handle_request
2022-09-20 09:48:23,174 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2022-09-

Exception: the request you have submitted is not valid. Mars server task finished in error; UserError: Restricted access to ERA5T. Please, check that your date selection is valid. For more information, visit https://climate.copernicus.eu/climate-reanalysis [mars]; Error code is -2; Request failed; Some errors reported (last error -2).

# 3) Converting the .grib file into .22 (NWS=6)

Set the path to where the .grib data was saved:

Open the .grib file:

In [4]:
path = pl.Path('/Users/tmiesse/work/libraries/adcirc-unswan/inputs')
file = 'download.grib'
grib = xr.open_dataset(path / file, engine='cfgrib')

2021-05-25 16:44:57,534 INFO ecCodes library found using name 'eccodes'.
2021-05-25 16:44:57,541 WARNING Ignoring index file '/Users/tmiesse/work/libraries/adcirc-unswan/inputs/download.grib.4cc40.idx' older than GRIB file
2021-05-25 16:45:02,361 INFO missing from GRIB stream: 'directionNumber'
2021-05-25 16:45:02,362 INFO missing from GRIB stream: 'frequencyNumber'
2021-05-25 16:45:02,712 INFO missing from GRIB stream: 'directionNumber'
2021-05-25 16:45:02,713 INFO missing from GRIB stream: 'frequencyNumber'
2021-05-25 16:45:02,943 INFO missing from GRIB stream: 'directionNumber'
2021-05-25 16:45:02,944 INFO missing from GRIB stream: 'frequencyNumber'


In [5]:
lat1,lat2 = 40,90
lon1,lon2 = -230, -90

lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = np.arange(0,len(grib.coords['time'][:]))
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(200, 560)

In [6]:
grib.coords['time'][:]

<xarray.DataArray 'time' (time: 228)>
array(['2011-11-01T00:00:00.000000000', '2011-11-01T02:00:00.000000000',
       '2011-11-01T04:00:00.000000000', ..., '2011-11-19T18:00:00.000000000',
       '2011-11-19T20:00:00.000000000', '2011-11-19T22:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 2011-11-01 ... 2011-11-19T22:00:00
    step        timedelta64[ns] ...
    surface     int64 ...
    valid_time  (time) datetime64[ns] ...
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time

In [5]:
lat[0]-lat[1]

0.25

In [8]:
lat1,lat2 = 40,90
lon1,lon2 = -230, -90

lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = np.arange(0,len(grib.coords['time'][:]))

#lat1,lat2 = 7,60
#lon1,lon2 = -100, -58

#lat = grib.coords['latitude'].values
#lon= grib.coords['longitude'].values
#time = np.arange(0,len(grib.coords['time'][:]))


temp,y,x = u.shape
u2,v2,p2 = [],[],[]
datay,datax = [], []
for t in time:
    for i in range(0,y):
        for ii in range(0,x):
            if (lat2>=lat[i]>lat1) and ((lon2+360)>=lon[ii]>(lon1+360)):
                u2.append(np.round(u[t,i,ii],3))
                v2.append(np.round(v[t,i,ii],3))
                p2.append(prmsl[t,i,ii])
                datay.append(i)
                datax.append(ii)

df = pd.DataFrame({'u':np.round(u2,2),'v':np.round(v2,2),'p':np.round(p2,2)})
df.to_csv('fort201510.22', sep='\t',index=False,header = False)  

temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1+360)]
len(temp),len(temp2)

(200, 560)

In [5]:
def find_columns(data):
    data2 = []
    for f in data.split(' '):
        if f != '':
            data2.append(f)   
    return data2

In [6]:
def f13_header(path,nodes,attributes:list,title='Spatial attributes description \n'):
    with open(pl.Path(path)/'fort.13','w') as fout:
        fout.write(title)
        fout.write(str(nodes)+'\n')
    
        for a in attributes:
            if 'mannings_n_at_sea_floor' in a:
                units = 'meters'
                nvalues = '1'
                default = '{:e}'.format(0.02)
            elif 'primitive_weighting_in_continuity_equation' in a:
                units = 'unitless'
                nvalues = '1'
                default = '{:e}'.format(0.03)
            elif 'surface_canopy_coefficient' in a:
                units = 'unitless'
                nvalues = '1'
                default = '{:e}'.format(1)                
            elif 'surface_directional_effective_roughness_length' in a:
                units = 'meters'
                nvalues = '12'
                default = '{:e} {:e} {:e} {:e} {:e} {:e} {:e} {:e} {:e} {:e} {:e} {:e}'.format(0,0,0,0,0,0,0,0,0,0,0,0)
                
            fout.write(a + '\n')
            fout.write(units+'\n')
            fout.write(nvalues+'\n')
            fout.write(default+'\n')      
    return print('generated fort.13')

In [9]:
file = nc4.Dataset('/Users/tmiesse/work/FHRL/arctic/model/high_tidal_062020/maxele.63.nc')

In [36]:
lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values-180
lons = [data-360 if (data>=0) else data for data in lon[:]]
time = grib.coords['time'].values[:]
sea_ice = grib.data_vars['siconc'].values[:]

In [12]:
file.variables['x']
file.variables['y']

KeyError: 'nodes'

In [ ]:
node = []
data = []
lat1,lat2 = 45,90
lon1,lon2 =-200, -108
for i in range(0,len(lon)):
    for ii in range(0,len(lat)):
        if (lat2>=lat[i]>lat1) and ((lon2)>=lons[ii]>(lon1)):
            lat0,lon0 = lat[ii],lon[i]
            node.append(ut.find_node_ak(file,lat0,lon0))
            data.append(sea_ice[201,i,ii])

In [ ]:
ut.find_node_ak(ncfile,geo.y,geo.x)

Extract the variables of interest:

In [7]:
#ice = grib.data_vars['siconc'].values[:]
u = grib.data_vars['u10'].values[:]
v = grib.data_vars['v10'].values[:]
prmsl = grib.data_vars['msl'].values[:]
lat = grib.coords['latitude'].values
lon= grib.coords['longitude'].values
time = grib.coords['time'].values[:]

Create boundaries that cover the extent of your mesh (in our case the North Atlantic and the Caribean):

In [17]:
lat1,lat2 = 44,88.75
lon1,lon2 = 130, 275
y = grib.coords['latitude'][:]
x = grib.coords['longitude'][:]
x = [data-360 if (data>=0) else data for data in x[:].values]
xgrid, ygrid = np.meshgrid(x,y)

In [21]:

ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]

/Users/tmiesse/miniconda3/envs/general/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


Create a grid in using the format expected for a NWS = 6 in 3 columns (uwind, vwind, pressure):

In [21]:
xgrid, ygrid = np.meshgrid(lon,lat)

In [2]:
root2 = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/joaquin3')
root = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/Jose')
start = '20170910 00:00:00'
freq = '0.5H'


In [23]:
t,s = 20,1
lat1,lat2 =37.148287,37.154156
lon1,lon2 =-75.946235, -75.938045

cmap = 'RdBu'
wl=[]
s = 0
grid_space = 50
ncfile = nc4.Dataset(root / 'cfs' / 'explicit_cd2' / 'swan_HS.63.nc')
ncfile2 = nc4.Dataset(root / 'cfs' / 'implicit015' / 'swan_HS.63.nc')
data = ncfile.variables['swan_HS'][:]
data2 = ncfile2.variables['swan_HS'][:]
ts2 = pd.date_range(start=start,periods=int(len(ncfile.variables['swan_HS'][:,30])),freq=freq)
time = ncfile.variables['time'][:]
node = ut.find_node_ak(ncfile,37.149152, -75.938702)
#xg = np.linspace(lon1-0.5,lon2+0.5,grid_space)
#yg = np.linspace(lat1-0.5,lat2+0.5,grid_space)
#ncfile = nc4.Dataset(root / 'cfs' / 'implicit0045' / 'fort.74.nc')
x,y = ncfile.variables['x'][:],ncfile.variables['y'][:]
#xgrid,ygrid = np.meshgrid(xg,yg)
limits = np.arange(-50,50,3)
cmaps = mpl.cm.get_cmap(cmap)  
#normalize = mpl.colors.Normalize(vmin=np.min(limits), vmax=np.max(limits))
#colors = [cmaps(normalize(value)) for value in limits]
gridvars = ncfile.variables      
var_element = 'element'
elems = gridvars[var_element][:,:]-1
triang = tri.Triangulation(x,y, triangles=elems)
for i in range(425,len(time[:445])):
    fig,ax = plt.subplots(figsize=(10,10))
    plt.rcParams["font.family"] = "Times New Roman"
    #prmsl= grib.data_vars['msl'].values[i,:,:]/100

    m = Basemap(projection='cyl',llcrnrlat=lat1,urcrnrlat=lat2,llcrnrlon=lon1,
                    urcrnrlon=lon2,resolution='h', epsg = 4269)
    m.arcgisimage(service='World_Imagery',xpixels=720, verbose= False,ax=ax)#service='Canvas/World_Light_Gray_Base',
    if data[i,:].mask.any():
        point_mask_indices = np.where(data[i,:].mask)
        tri_mask = np.any(np.in1d(elems, point_mask_indices).reshape(-1, 3), axis=1)
        triang.set_mask(tri_mask)
    #m.drawcoastlines()
    data3 = (data[i,:])/np.max(data[i,node])*100
    data4 = (data2[i,:])/np.max(data2[i,node])*100
    m.drawparallels(np.arange(0, 100, 0.005), linewidth=0.01,labels=[1,0,0,0], color='#595959')
    m.drawmeridians(np.arange(0, 360, 0.005), linewidth=0.01, labels=[0,0,0,1],zorder=3)
    contour = plt.tricontourf(triang,(data3.data-data4.data),levels=limits,cmap=cmap,vmin=np.min(limits),vmax=np.max(limits),alpha=0.6,aspect='auto',extend='both')
    #ax.quiver(xgrid,ygrid,ugrid,vgrid, pivot='mid', scale = 600, color='w')
    #cax, _ = mpl.colorbar.make_axes(ax,shrink=0.75)
    #cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)
    cbar = plt.colorbar(contour,ax=ax,shrink=0.5,boundaries=limits)
    cbar.set_label(r'Relative Difference $H_{S}$ [%]',fontsize=12)
    contour.set_clim(np.min(limits),np.max(limits))
    file_number = '%05d'%i
    wl.append('WL{}.png'.format(file_number))
   
    #cbar.mappable.set_clim(np.min(limits),np.max(limits))
    #cbar.ax.tick_params(labelsize=12)
    #fig.suptitle(f'Ice',x=0.435,y=0.725,fontsize=16)
    ax.set_xlabel('\nDate:{}'.format(ts2[i]),position=(0.5,-1.5),fontsize=14)
    #plt.show()
    plt.savefig('WL{}.png'.format(file_number),dpi=400,
                        bbox_inches = 'tight', pad_inches = 0.1)
    plt.close()
images = []
for ii in range(0,len(wl)):
    frames = Image.open(wl[ii])
    images.append(frames)
images[0].save('jose_difference_hs.gif',
   save_all=True,
   append_images=images[1:],
   delay=.1,
   duration=200,
   loop=0)
for f in glob.glob('WL*'):
    os.remove(f)   

In [22]:
#data3-
data4

masked_array(data=[--, --, --, ..., --, --, --],
             mask=[ True,  True,  True, ...,  True,  True,  True],
       fill_value=-99999.0)

In [7]:
root2 = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/joaquin3')
root = pl.Path('/Users/tmiesse/work/FHRL/eeslr/modelling/Jose')
#start = '20170910 00:00:00'
start = '20150918 01:00:00'
freq = '0.5H'
t,s = 20,1
lat1,lat2 =37.118562,37.123928
lon1,lon2 =-75.966771, -75.957137

cmap = 'jet'
#key = keys[6]
wl=[]
s = 0
grid_space = 50
ncfile = nc4.Dataset(root2 / 'explicit_cd1' / 'swan_HS.63.nc')
data = ncfile.variables['swan_HS'][:]
ts2 = pd.date_range(start=start,periods=int(len(ncfile.variables['swan_HS'][:,30])),freq=freq)
time = ncfile.variables['time'][:]
#xg = np.linspace(lon1-0.5,lon2+0.5,grid_space)
#yg = np.linspace(lat1-0.5,lat2+0.5,grid_space)
#ncfile = nc4.Dataset(root / 'cfs' / 'implicit0045' / 'fort.74.nc')
x,y = ncfile.variables['x'][:],ncfile.variables['y'][:]
#xgrid,ygrid = np.meshgrid(xg,yg)
limits = np.arange(0,1.5,0.1)
cmaps = mpl.cm.get_cmap(cmap)  
#normalize = mpl.colors.Normalize(vmin=np.min(limits), vmax=np.max(limits))
#colors = [cmaps(normalize(value)) for value in limits]
gridvars = ncfile.variables      
var_element = 'element'
elems = gridvars[var_element][:,:]-1
triang = tri.Triangulation(x,y, triangles=elems)
for i in range(410,len(time[:460])):
    fig,ax = plt.subplots(figsize=(10,10))
    plt.rcParams["font.family"] = "Times New Roman"
    #prmsl= grib.data_vars['msl'].values[i,:,:]/100

    m = Basemap(projection='cyl',llcrnrlat=lat1,urcrnrlat=lat2,llcrnrlon=lon1,
                    urcrnrlon=lon2,resolution='h', epsg = 4269)
    m.arcgisimage(service='World_Imagery',xpixels=600, verbose= False,ax=ax)#service='Canvas/World_Light_Gray_Base',
    if data[i,:].mask.any():
        point_mask_indices = np.where(data[i,:].mask)
        tri_mask = np.any(np.in1d(elems, point_mask_indices).reshape(-1, 3), axis=1)
        triang.set_mask(tri_mask)
    #m.drawcoastlines()
    m.drawparallels(np.arange(0, 100, 0.005), linewidth=0.01,labels=[1,0,0,0], color='#595959')
    m.drawmeridians(np.arange(0, 360, 0.005), linewidth=0.01, labels=[0,0,0,1],zorder=3)
    contour = plt.tricontourf(triang,data[i,:],levels=limits,cmap=cmap,vmin=np.min(limits),vmax=np.max(limits),alpha=0.6,aspect='auto')
    #ax.quiver(xgrid,ygrid,ugrid,vgrid, pivot='mid', scale = 600, color='w')
    #cax, _ = mpl.colorbar.make_axes(ax,shrink=0.75)
    #cbar = mpl.colorbar.ColorbarBase(cax, cmap=cmap)
    cbar = plt.colorbar(contour,ax=ax,shrink=0.5,boundaries=limits)
    cbar.set_label(r'$H_{S}$ [m]',fontsize=12)
    contour.set_clim(np.min(limits),np.max(limits))
    file_number = '%05d'%i
    wl.append('WL{}.png'.format(file_number))
   
    #cbar.mappable.set_clim(np.min(limits),np.max(limits))
    #cbar.ax.tick_params(labelsize=12)
    #fig.suptitle(f'Ice',x=0.435,y=0.725,fontsize=16)
    ax.set_xlabel('\nDate:{}'.format(ts2[i]),position=(0.5,-1.5),fontsize=14)
    #plt.show()
    plt.savefig('WL{}.png'.format(file_number),dpi=400,
                        bbox_inches = 'tight', pad_inches = 0.1)
    plt.close()
images = []
for ii in range(0,len(wl)):
    frames = Image.open(wl[ii])
    images.append(frames)
images[0].save('joaquin_hs_implicit.gif',
   save_all=True,
   append_images=images[1:],
   delay=.1,
   duration=200,
   loop=0)
for f in glob.glob('WL*'):
    os.remove(f)   

In [10]:
idx = np.where((lon1 >= lon)&(lon<=lon2))[0]
idy = np.where((lat1 >= lat)&(lat<=lat2))[0]

Start populating the grid with data from the .grib file:

In [34]:
def force2owi(file,dx,dy,dt,ilon,ilat,start,end,swlat,swlon,header,grib,lat1,lat2,lon1,lon2):
    if file == 'fort.222':
        u = grib.data_vars['u10'].values[:]
        v = grib.data_vars['v10'].values[:]
    elif file == 'fort.221':
        prmsl = grib.data_vars['msl'].values[:]
    elif file == 'fort.225':
        ice = grib.data_vars['siconc'].values[:]
    else:
        raise TypeError('not correct forcing format')
    
    
    lat = grib.coords['latitude'].values
    lon= grib.coords['longitude'].values
    idx = np.where((lon1 >= lon)&(lon<=lon2))[0]
    idy = np.where((lat1 >= lat)&(lat<=lat2))[0]
    with open(file,'w') as fin:
        fin.write(header)
        for t in range(len(dt[:])):
            param ='iLat= {}iLong= {}DX={:.4f}DY={:.4f}SWLat={:.4f}SWLon={:.4f}DT={} \n'.format(ilat,ilon,dx,dy,swlat,swlon,str(dt[t].strftime('%Y%m%d%H%M')))
            fin.write(param)
            i = 0
            data = []
            d1,d2 = [],[] 
            for y in idy[:]:
                for x in idx[:]:
                    if file == 'fort.222':
                        windx = u[t,y,x]
                        windy = v[t,y,x]
                        d1.append(' {:{width}.{prec}f}'.format(windx,width=9,prec=4))
                        d2.append(' {:{width}.{prec}f}'.format(windy,width=9,prec=4))
                    elif file == 'fort.221':
                        press = prmsl[t,y,x]/100
                        data.append(' {:{width}.{prec}f}'.format(press,width=6,prec=4))
                    #elif file == 'fort.225':
                        #sea = ice[t,y,x]
                        #if np.isnan(sea):
                        #    sea = -1.0
                        #else:
                        #    sea =sea * 100
                        #data.append(' {:{width}.{prec}f}'.format(sea,width=9,prec=4))
            i = 0
            if file == 'fort.222':
                for d in range(len(d1)):
                    if (i == 7) :
                        fin.write(d1[d]+'\n')
                        i=0
                    elif (d+1>=len(d1)):
                        fin.write(d2[d]+'\n')
                        i = 0
                        for d in range(len(d2)):
                            if (i == 7) :
                                fin.write(d2[d]+'\n')
                                i=0
                            elif (d+1>=len(d2)):
                                fin.write(d2[d]+'\n')
                            else:
                                fin.write(d2[d])
                                i +=1
                    else:
                        fin.write(d1[d])
                        i +=1
            else:
                for d in range(len(data)):
                    if (i == 7) :
                        fin.write(data[d]+'\n')
                        i=0
                    elif (d+1>=len(data)):
                        fin.write(data[d]+'\n')
                    else:
                        fin.write(data[d])
                        i +=1
    return print('generated owi formatted forcings')

In [35]:
ts = [(t - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')  for t in time]
ts2 = [datetime.datetime.utcfromtimestamp(t) for t in ts]
dx = lon[1]-lon[0]
dy = lat[0]-lat[1]
ilon = len(idx)
ilat = len(idy)
start = ts2[0]
end = ts2[-1]
swlat = lat1
swlon = -230.0
header = 'FHRL tmiesse wind Format                               {}     {}\n'.format(str(start.strftime('%Y%m%d%H')),str(end.strftime('%Y%m%d%H')))#     {}'
dt = ts2
file = 'fort.222'

/Users/tmiesse/miniconda3/envs/general/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


In [37]:
force2owi(file,dx,dy,dt[:],ilon,ilat,start,end,swlat,swlon,header,grib,lat1,lat2,lon1,lon2)


generated owi formatted forcings


In [40]:
fprintf(sys.stdout, "bagel %d donut %f", 6, 3.1459)

bagel 6 donut 3.145900

Repeat steps 1, 2, and 3 for the second part of the storm

Make sure that when you repeat the process for the other part of the storm (07/01/2014 to 07/09/2014) you change the name of the output file.

Use a text editor (Sublime for instance) to copy the 'arthur_2.22' file at the end of the 'arthur_1.22' file. This complete file refers to the whole storm.

Check the extent of your grid:

In [10]:
temp = lat[(lat2>=lat) & (lat>lat1)]
temp2 = lon[(lon2+360>=lon) & (lon>lon1)]
len(temp),len(temp2)

(174, 1440)

# Updating the fort.15 file:

Change the line YYYY MM DD HH24 to:

212 168 60.0 -100.0 0.25 0.25 21600 3600 ! YYYY MM DD HH24 StormNumber BLAdj 

where, 212 is the number of values in Y, 168 is the number of values in X, 60 is the maximum Y, -100 is the minimum X, 0.25 are the X and Y increments, and 21600 is the time interval in seconds (6hours).